# Simulation at different carbon source uptakes

Code for reproducing panels of Figure 4

## Setup environment

In [ ]:
import coralme
from coralme.builder.helper_functions import *
import pickle
import pandas
import os
import tqdm
import json
import numpy
import sympy

## Functions to load models

In [ ]:
def load_me(filename='me_model.pickle'):
    with open(filename, "rb") as f:
        return pickle.load(f)
    
def get_org_dirs(directory,files=False):
    if files:
        return [i for i in os.listdir(directory) if '.pkl' in i]
    return [i for i in os.listdir(directory) if os.path.isdir(directory+i) and '.' not in i and '_' not in i]

def run(i,directory,files=False,step=2,solve=False):
    if files:
        modelpath = directory + i
    elif step == 2:
        modelpath = directory + '{}/MEModel-step2-{}.pkl'.format(i,i)
    elif step == 3:
        modelpath = directory + '{}/MEModel-step3-{}-TS.pkl'.format(i,i)
    model = load_me(modelpath)
    if solve:
        model.optimize()
    return i.split(".pkl")[0], model

def load_models_from_directory(directory,solve=False, step = 2, files=False):
    d = {}
    dirs = get_org_dirs(directory,files=files)
    for org in tqdm.tqdm(dirs):
        args = [org,directory]
        kwds = {'files' : files, 'step' : step, 'solve' : solve}
        i,model = run(*args,**kwds)
        d[i] = model
    return d

## Load dME-models in directory

In [ ]:
models = {
    'clean':load_models_from_directory('./clean/',step=3)
         }

In [ ]:
# models={}
# models["clean"] = {"pputida":coralme.io.pickle.load_pickle_me_model("./clean/pputida/MEModel-step3-pputida-TS.pkl")}

## Save base fluxes

In [ ]:
flux_dict = {}
for org,model in models["clean"].items():
    flux_dict[org] = model.solution.fluxes
pandas.DataFrame.from_dict(flux_dict).to_csv("./analysis/simulations_base.csv")

## Get carbon sources

In [ ]:
def get_exchange(reactions):
    l = []
    for r in reactions:
        if r.flux >= -1e-3:
            continue
        met = next(i for i in r.reactants)
        if "C" not in met.elements:
            continue
        l.append(r)
    return l

def get_carbon_sources(model):
    d = {}
    reactions = get_exchange(model.reactions.query("^EX_|^TS_"))
    for r in reactions:
        met = next(i for i in r.reactants)
        d[r] = r.flux * met.elements["C"]
    return d

In [ ]:
carbon_sources = {}
for org,model in tqdm.tqdm(models['clean'].items()):
    carbon_sources[org] = get_carbon_sources(model)

## Open all previously fixed bounds

In [ ]:
def open_bounds(model):
    for r in model.reactions.query("^EX_"):
        lb,ub = r.bounds
        if ub < 0 or lb > 0 or (ub and ub == lb) or (ub and ub < 1000) or (lb and lb > -1000):
            nu = 1000 if ub > 0 else 0
            nl = -1000 if lb < 0 else 0
            print(r, r.bounds, (nl,nu))
            r.bounds = (nl,nu)

for org, model in models['clean'].items():
    print(org)
    open_bounds(model)    
    print()

## Calculate maximum rates

### Calculate

In [ ]:
def run(org):
    for r,d in carbon_sources[org].items():
        r.bounds = (-1000,0)
    
    model = models['clean'][org]
    model.optimize(tolerance = 1e-6,verbose=False)
    if hasattr(model,"solution"):
        solution = model.solution.fluxes
    else:
        solution = {k.id:0 for k in model.reactions}
    return org, solution

In [ ]:
models_to_run = list(models['clean'].keys())

In [ ]:
import multiprocessing as mp
NP = min([12,len(models_to_run)])
pool = mp.Pool(NP)
pbar = tqdm.tqdm(total=len(models_to_run),position=0,leave=True)
pbar.set_description('Running ({} threads)'.format(NP))

flux_dict = {}
def collect_result(result):
    pbar.update(1)
    flux_dict[result[0]] = result[1]
for org in models_to_run:
    args = ([org])
    kwds = {}
    pool.apply_async(run,args, kwds, callback=collect_result)
pool.close()
pool.join()

In [ ]:
pandas.DataFrame.from_dict(flux_dict).to_csv("./analysis/simulations_open.csv")

### Load

In [ ]:
df = pandas.read_csv("./analysis/simulations_open.csv",index_col = 0)
flux_dict = {}
for org in df.columns:
    flux_dict[org] = df[org].dropna().to_dict()

## Setup simulation ranges

In [ ]:
ranges = {}
steps = 10
steps_after_max = 5
for org in tqdm.tqdm(models_to_run):
    orgd = carbon_sources[org]
    ranges[org] = {}
    for r in orgd:
        minimum = 0
        maximum = flux_dict[org][r.id]
        if minimum == maximum:
            continue
        delta = (maximum-minimum)/steps
        ranges[org][r] = numpy.arange(minimum+delta,maximum+steps_after_max*delta, delta)

In [ ]:
l = []
for org in models_to_run:
    for r in ranges[org]:
        l.append(len(ranges[org][r]))

In [ ]:
length = min(l)
length

In [ ]:
directory = './analysis/{}steps/'.format(str(steps))
if not os.path.exists(directory):
    os.makedirs(directory)

## Simulate

In [ ]:
import copy
def run(org):
    results = {}
    model = models['clean'][org]
#     model = copy.deepcopy(model0)
    for idx in range(length):
        for r in ranges[org]:
            const = ranges[org][r][idx]
            r.bounds = (const,0)
        model.optimize(tolerance = 1e-6, verbose = False)
        if not hasattr(model,"solution") or model.solution.status != "optimal":
            results[idx] = {k.id:0 for k in model.reactions}
        else:
            results[idx] = model.solution.fluxes
    return org,results

In [ ]:
results_dict = {}

In [ ]:
import multiprocessing as mp
NP = min([12,len(models_to_run)])
pool = mp.Pool(NP)
pbar = tqdm.tqdm(total=len(models_to_run),position=0,leave=True)
pbar.set_description('Running ({} threads)'.format(NP))

def collect_result(result):
    pbar.update(1)
    results_dict[result[0]] = result[1]

for org in models_to_run:
    args = ([org])
    pool.apply_async(run,args, callback=collect_result)
pool.close()
pool.join()

In [ ]:
results_dict["pputida"]

## Save

In [ ]:
for org,result in results_dict.items():
    result = pandas.DataFrame.from_dict(result)
    result.to_csv(directory + "{}_C_uptake.csv".format(org))